In [1]:
#importing Libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
from matplotlib import cm

import plotly.express as px
import seaborn as sns
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)    #THIS LINE IS MOST IMPORTANT AS THIS WILL DISPLAY PLOT ON 
#NOTEBOOK WHILE KERNEL IS RUNNING
import plotly.graph_objects as go
import requests
import pandas as pd
import json


In [2]:



# Set API endpoint and parameters
url = 'https://data.telangana.gov.in/api/1/metastore/schemas/dataset/items/ffbdd40f-573f-4160-9bea-3fba961834c5?show-reference-ids=true'

response = requests.get(url)

# Check if request was successful (status code 200 means success)
if response.status_code == 200:
    # Get JSON data from response
    data = response.text
    parse_json = json.loads(data)
    
   
    

    

In [3]:
#runtime 2m

import csv

import concurrent.futures

# Generate the list of download URLs using list comprehension
download_urls = [parse_json["distribution"][i]['data']['downloadURL'] for i in range(0, 34)]

# Use the requests.Session object to reuse the same TCP connection for all requests
session = requests.Session()

# Create separate CSV files for 13 and 16 columns
with open('data_13.csv', 'w', newline='') as csvfile_13, open('data_16.csv', 'w', newline='') as csvfile_16:
    writer_13 = csv.writer(csvfile_13)
    writer_16 = csv.writer(csvfile_16)
    
    # Use flag variables to check if headers have been written already
    header_13_written = False
    header_16_written = False

    # Use multiprocessing to download and process the CSV data in parallel
    with concurrent.futures.ProcessPoolExecutor() as executor:
        responses = executor.map(session.get, download_urls)

    # Read data from response as CSV and write to separate CSVs based on the number of columns
    for response in responses:
        reader = csv.reader(response.text.strip().splitlines())
        header = next(reader)
        if len(header) == 13:
            if not header_13_written:
                writer_13.writerow(header)
                header_13_written = True
            for row in reader:
                writer_13.writerow(row)
        elif len(header) == 16:
            if not header_16_written:
                writer_16.writerow(header)
                header_16_written = True
            for row in reader:
                writer_16.writerow(row)

# Merge data from both CSV files into a single dataframe using pandas
df_13 = pd.read_csv('data_13.csv')
df_16 = pd.read_csv('data_16.csv')
df_combined = pd.concat([df_13, df_16], axis=0)

# Write merged data to a single CSV file
df_combined.to_csv('combined_data.csv', index=False)


In [4]:

df = pd.read_csv("combined_data.csv", low_memory=False)

In [5]:
df.columns

Index(['DDate', 'AmcCode', 'AmcName', 'YardCode', 'YardName', 'CommCode',
       'CommName', 'VarityCode', 'VarityName', 'Arrivals', 'Minimum',
       'Maximum', 'Model', 'ProgArrivals', 'Valuation', 'MarketFee'],
      dtype='object')

In [6]:

# Remove three columns with 0 values
df = df.drop(['ProgArrivals', 'Valuation','MarketFee'], axis=1)

df['DDate'] = df['DDate'].str.strip()
#getting release date and movie length from movie_movierelease
new = df["DDate"].str.split(' ', n = 1, expand = True)  #splitting the values into three

# making separate release date column from new data frame 
df["date"]= new[0] 
  
df['date'] = pd.DatetimeIndex(df['date'])
df = df.drop('DDate', axis=1)
df = df.set_index('date')
df = df.reset_index()
df.head()

#Alternative Method but takes more time
# # Convert the datetime strings in the 'DDate' column to datetime objects
# df['DDate'] = pd.to_datetime(df['DDate'], format='%Y-%m-%d %H:%M:%S %z')

# # Format the datetime objects to a string with the desired format
# df['DDate'] = df['DDate'].dt.strftime('%d/%m/%y')

# # Convert the 'DDate' column back to datetime format for sorting
# df['DDate'] = pd.to_datetime(df['DDate'], format='%d/%m/%y')

# # Sort the DataFrame by 'DDate' column and reset the index
# df = df.sort_values(by='DDate').reset_index(drop=True)




,date,AmcCode,AmcName,YardCode,YardName,CommCode,CommName,VarityCode,VarityName,Arrivals,Minimum,Maximum,Model
0,2014-01-02,246,Warangal,886,Warangal,1,Paddy,1,Common,637.0,1310.0,1330.0,1320.0
1,2014-01-02,221,Jammikunta,36,Jammikunta,1,Paddy,240,Common (1001),161.0,1300.0,1310.0,1310.0
2,2014-01-03,246,Warangal,886,Warangal,1,Paddy,1,Common,287.0,1310.0,1330.0,1320.0
3,2014-01-04,246,Warangal,886,Warangal,1,Paddy,1,Common,329.0,1310.0,1330.0,1320.0
4,2014-01-06,246,Warangal,886,Warangal,1,Paddy,1,Common,149.0,1500.0,1675.0,1600.0


In [8]:
df.to_csv("Tel.csv")